In [2]:
import pandas as pd
# !pip install pandas_datareader
from pandas_datareader import data as web
from datetime import datetime
from datetime import date
import investpy as inv
import numpy as np


C:\Users\583152\AppData\Roaming\Python\Python37\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [3]:
#ler massa de dados
dreitr = pd.read_csv('DADOS/itr_cia_aberta_DRE_con_2011-2021')
dredfp = pd.read_csv('DADOS/dfp_cia_aberta_DRE_con_2011-2021')

# relação de empresas
empresas = dreitr[['DENOM_CIA','CD_CVM','CNPJ_CIA']].drop_duplicates().set_index('CD_CVM')
# Relação de contas
# contas = empresa[['CD_CONTA','DS_CONTA']].drop_duplicates().set_index('CD_CONTA')
# '3.99.01.01'


# Manter uma relação de datas padrão
data_ini = '01/01/2011'
data_fim = '09/02/2022'



In [4]:
def CalculoLPA(codigo_cvm,conta_cvm='3.99.01.01'):

    #Selecao de empresas nos arquivos principais
    empresa = dreitr[dreitr['CD_CVM']==codigo_cvm]
    empresadfp = dredfp[dredfp['CD_CVM']==codigo_cvm]

    #LPA do ITR
    lpa = empresa[empresa['CD_CONTA'] == conta_cvm]
    lpa = lpa[lpa['ORDEM_EXERC'] == 'ÚLTIMO']
    lpa['DT_INI_EXERC'] = pd.to_datetime(lpa['DT_INI_EXERC'])
    lpa['DT_FIM_EXERC'] = pd.to_datetime(lpa['DT_FIM_EXERC'])
    lpa['Timedelta'] = pd.to_timedelta(lpa['DT_FIM_EXERC']-lpa['DT_INI_EXERC'], unit='D', errors='raise') #calcular o delta dos registros para localizar os registros apenas de trimestres separados e não de acumulado do ano
    lpa = lpa[lpa['Timedelta'] < pd.Timedelta(100, unit='D')] #filtrar somente registros de trimestres em separado
    lpa = lpa[['DT_FIM_EXERC','VL_CONTA']].set_index('DT_FIM_EXERC')

    #LPA do DFP
    lpadfp = empresadfp[empresadfp['ORDEM_EXERC'] == 'ÚLTIMO']
    lpadfp = lpadfp[lpadfp['CD_CONTA'] == conta_cvm]
    lpadfp['DT_INI_EXERC'] = pd.to_datetime(lpadfp['DT_INI_EXERC'])
    lpadfp['DT_FIM_EXERC'] = pd.to_datetime(lpadfp['DT_FIM_EXERC'])
    lpadfp = lpadfp[['DT_FIM_EXERC','VL_CONTA']].set_index('DT_FIM_EXERC')
    lpa['ano'] = lpa.index.year #testar remover linha
    lpadfp['ano'] = lpadfp.index.year


    #Obter lpa somente do quarto tri
    lpaano = lpa.groupby('ano').sum()
    lpaano.rename(columns={'VL_CONTA':'VL_CONTA_AC_3Q'}, inplace=True)
    lpadfpano = lpadfp.groupby('ano').sum()
    lpadfpano.rename(columns={'VL_CONTA':'VL_CONTA_AC_4Q'}, inplace=True)

    lpa4q = lpaano.join(lpadfpano)
    lpa4q['VL_CONTA_SO_4Q'] = lpa4q['VL_CONTA_AC_4Q'] - lpa4q['VL_CONTA_AC_3Q']
    lpa4q.reset_index(inplace=True)
    lpa4q['date'] = lpa4q['ano'].astype(str)+'-12-31'
    lpa4q = lpa4q[['date','ano','VL_CONTA_SO_4Q']].set_index('date')
    lpa4q.rename(columns={'VL_CONTA_SO_4Q':'VL_CONTA'}, inplace=True)

    #Consolidar dataframe com LPAs por trimestre
    lpa_trimestral = pd.concat([lpa, lpa4q]).drop('ano',axis=1)
    lpa_trimestral.index = pd.to_datetime(lpa_trimestral.index)
    lpa_trimestral.sort_index(inplace=True)

    lpa_trimestral['ultimos 4q'] = lpa_trimestral['VL_CONTA'].rolling(4, min_periods=4).sum()
    lpa_trimestral['media 4q'] = lpa_trimestral['VL_CONTA'].rolling(4, min_periods=4).mean()

    lpa_trimestral['CD_CVM'] = codigo_cvm
    lpa_trimestral['CD_CONTA'] = conta_cvm


    # Obter ticker b3

    global ticker

    ticker = pd.read_excel('AUXILIAR\Empresas_Cadastro_Full.xlsx')
    tickerPN = ticker[ticker['CD_CVM'] == codigo_cvm]['PN'].iloc[0]
    tickerON = ticker[ticker['CD_CVM'] == codigo_cvm]['ON'].iloc[0]


    
    if conta_cvm == '3.99.01.01': #ON
        ticker=tickerON
    elif conta_cvm == '3.99.01.02': #PN
        ticker=tickerPN


   
    lpa_trimestral['TP_TICKER'] = ticker  #talvez daria para levar o nome do ticker somente no arquivo salvo, nao precisaria ter uma serie repetindo               

    lpa_trimestral.index.name = 'Date'

    # Obter cotações do periodo  
    
    if pd.isna(ticker) == False:
        cotacoes = pd.DataFrame(inv.get_stock_historical_data(stock=ticker,
                                            country='brazil',
                                            from_date=data_ini,
                                            to_date=data_fim)['Close'])

        # cotacoes = cotacoes.join(lpa_trimestral)

        df_output = pd.date_range(start=data_ini,end=data_fim)
        df_output = pd.DataFrame(df_output, columns=['Date']).set_index('Date')
        df_output = df_output.join(cotacoes).join(lpa_trimestral)
    else:
        df_output = pd.DataFrame(columns=['ticker not found'])


    return df_output.ffill()


In [5]:
CalculoLPA(14451,'3.99.01.01')

,Close,VL_CONTA,ultimos 4q,media 4q,CD_CVM,CD_CONTA,TP_TICKER
Date,,,,,,,
2011-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-03,3.75,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-04,3.75,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-05,3.75,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2022-08-29,15.50,2.0845,113.8712,28.4678,14451.0,3.99.01.01,CEBR3
2022-08-30,15.50,2.0845,113.8712,28.4678,14451.0,3.99.01.01,CEBR3
2022-08-31,15.50,2.0845,113.8712,28.4678,14451.0,3.99.01.01,CEBR3


In [6]:
# gerar todas as planilhas de LPA

t = pd.read_excel('AUXILIAR\Empresas_Cadastro_Full.xlsx')

for empr in list(empresas.index):
    # print(empr)
    # print(ticker)
    tPN = t[t['CD_CVM'] == empr]['PN'].iloc[0]
    if pd.isna(tPN) == False:
        
        
        df = CalculoLPA(empr,'3.99.01.02')
        df.to_excel('LPA/lpa_'+str(tPN)+'.xlsx')

    tON = t[t['CD_CVM'] == empr]['ON'].iloc[0]
    if pd.isna(tON) == False:

        df = CalculoLPA(empr,'3.99.01.01')
        df.to_excel('LPA/lpa_'+str(tON)+'.xlsx')








In [7]:
lx = list(empresas.index)

In [8]:
empr

4693

In [9]:
pd.isna(tPN)

True

In [10]:
empr

4693

In [11]:
empresas.loc[14451]

DENOM_CIA    CIA ENERGETICA DE BRASILIA
CNPJ_CIA             00.070.698/0001-11
Name: 14451, dtype: object

In [12]:
empresas_full = pd.read_excel('AUXILIAR\Empresas_Cadastro_Full.xlsx')

In [13]:
empresas_full[empresas_full['CD_CVM'] == 14451]

,Unnamed: 0,CD_CVM,DENOM_CIA,CNPJ_CIA,Sigla,SETOR ECONÔMICO,SUBSETOR,SEGMENTO,LISTAGEM,ON,PN,ON.1,PN.1,ON.2,PN.2
1,1,14451,CIA ENERGETICA DE BRASILIA,00.070.698/0001-11,CEBR,Utilidade Pública,Energia Elétrica,CEB,CEBR,CEBR3,NaN,CEBR3,NaN,CEBR3,NaN


In [14]:
ticker = pd.read_excel('AUXILIAR\Empresas_Cadastro_Full.xlsx')
tickerPN = ticker[ticker['CD_CVM'] == 14451]['PN'].iloc[0]
tickerON = ticker[ticker['CD_CVM'] == 14451]['ON'].iloc[0]

In [15]:
pd.isna(tickerPN)

True

In [16]:
tickerON

'CEBR3'

In [17]:
pd.isna(tickerPN) == False

False